In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.util import makeMovie

In [5]:
import subprocess
output_name = '20170810'
folder_name = '20170809-4'

# vmem読み込み

In [10]:
vmem1 = RawCam(path='/mnt/recordings/SimulationResults/20170804-1', cam_type='numpy', image_width=300, image_height=300, frame_start=0, frame_end=-1)
vmem2 = RawCam(path='/mnt/recordings/SimulationResults/20170804-3', cam_type='numpy', image_width=300, image_height=300, frame_start=0, frame_end=-1)
vmem1.data = np.vstack((vmem1.data, vmem2.data[1:, :, :]))
vmem_temp = VmemMap(vmem1)
vmem1.cmap = vmem_temp.cmap
vmem1.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/%s' % (output_name, folder_name))
subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/movie.mp4"'.format(output_name, folder_name), shell=True)

0

In [6]:
vmem1 = RawCam(path='/mnt/recordings/SimulationResults/%s' % folder_name, cam_type='numpy', image_width=300, image_height=300, frame_start=0, frame_end=-1)
vmem_temp = VmemMap(vmem1)
vmem1.cmap = vmem_temp.cmap
vmem1.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/%s' % (output_name, folder_name))
subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/movie.mp4"'.format(output_name, folder_name), shell=True)

0

# 位相分散解析

In [29]:
output_name = '20170810'
folder_name = '20170810-3'
vmem = RawCam(path='/mnt/recordings/SimulationResults/%s' % folder_name, cam_type='numpy', image_width=300, image_height=300, frame_start=0, frame_end=-1)
vmem_temp = VmemMap(vmem)
vmem.cmap = vmem_temp.cmap
vmem.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/%s/vmem' % (output_name, folder_name))
pmap = PhaseMap(vmem, shrink=2)
pmap.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/%s/pmap' % (output_name, folder_name))
pvmap = PhaseVarianceMap(pmap, size=3)
pvmap.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/%s/pvmap' % (output_name, folder_name))

subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/vmem/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/vmem/vmem.mp4"'.format(output_name, folder_name), shell=True)
subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/pmap/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/pmap/pmap.mp4"'.format(output_name, folder_name), shell=True)
subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/pvmap/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/pvmap/pvmap.mp4"'.format(output_name, folder_name), shell=True)

0

# hjmap & Inamap & pvmap

In [30]:
hj_array = np.zeros((401, 300, 300))
m_array = np.zeros((401, 300, 300))
Ina = np.zeros((401, 300, 300))

In [31]:
for i in range(400):
    num = '{0:04d}'.format(i)
    a = np.load('/mnt/recordings/SimulationResults/%s/cell_%s/%s' % (folder_name, num, 'h.npy'))
    b = np.load('/mnt/recordings/SimulationResults/%s/cell_%s/%s' % (folder_name, num, 'j.npy'))
    hj_array[i, :, :] = a*b
    m_array[i, :, :] = np.load('/mnt/recordings/SimulationResults/%s/cell_%s/%s' % (folder_name, num, 'm.npy'))
    Ina[i, :, :] = np.load('/mnt/recordings/SimulationResults/%s/cell_%s/%s' % (folder_name, num, 'xina.npy'))

In [ ]:
width = 10
thres = 0.05

save_dir = '/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20170810/%s/contour' % folder_name

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

x = np.arange(300)
y = np.arange(300)
X, Y = np.meshgrid(x, y)


for i in range(600):
    if i < width+1:
        continue
    
    plt.clf()
    
    im_out = np.zeros([300, 300, 3], dtype=np.uint8)
    
    im_out[:, :, 0] = (np.sum((Ina[i-width:i, :, :] < -1), axis=0) > 0) * 255
    
    ans = np.zeros((1, 300, 300))
    for j in range(width):
        ans += ((hj_array[i+j-1-width:i+j-width, :, :] < thres) * (hj_array[i+j-width:i+j+1-width, :, :] >= thres))
        
    im_out[:, :, 1] = (ans[0, :, :] > 0) * 255
    im_out[:, :, 2] = (ans[0, :, :] > 0) * 255

    
    #plt.imsave(os.path.join(save_dir, 'wfwt_{0:0>6}.png'.format(i-width-1)), im_out)    
    plt.subplot(1,2,1)
    plt.imshow(im_out)
    
    
    Z = hj_array[i,:,:] 
    Z = Z[::-1, :]
    plt.subplot(1,2,2)
    plt.contour(X, Y, Z)
    
    plt.gca().set_aspect('equal')    
    plt.savefig(os.path.join(save_dir, '{0:0>6}.png'.format(i-width-1)))

In [ ]:
subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/contour/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/{1}/contour/contour.mp4"'.format(output_name, folder_name), shell=True)